Deep Residual Learning for Image Recognition

In [6]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [7]:
def Identity_shortcut(x,filters,strides = 1):
    shortcut = x
    x = layers.Conv2D(filters = filters, kernel_size = 3, strides = strides, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters = filters, kernel_size = 3, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    if strides != 1: # zero-padding
        shortcut = layers.MaxPool2D(pool_size = 2, strides=2)(shortcut)
        shortcut = tf.pad(shortcut, tf.constant([[0, 0,], [0, 0,], [0, 0,], [0, filters//2]]), "CONSTANT")
    x = layers.Add()([shortcut,x])
    x = layers.ReLU()(x)
    return x

def Projection_shortcut(x,filters,strides = 1):
    shortcut = x
    x = layers.Conv2D(filters = filters, kernel_size = 3, strides = strides, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters = filters, kernel_size = 3, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    if strides != 1: # 1x1 convolution
        shortcut = layers.Conv2D(filters = filters, kernel_size = 1, strides = strides, 
                                padding = 'same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Add()([shortcut,x])
    x = layers.ReLU()(x)
    return x

def resnet_20(shape = (224,224,3)):
    inputs = layers.Input(shape=shape)
    x = layers.Conv2D(filters = 16, kernel_size = 3,strides = 1,padding = 'same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    #x = layers.MaxPooling2d(pool_size = (3,3), strides = 2, padding = 'same')
    
    x = Identity_shortcut(x,16)
    x = Identity_shortcut(x,16)
    x = Identity_shortcut(x,16)
    
    x = Identity_shortcut(x,32,2)
    x = Identity_shortcut(x,32)
    x = Identity_shortcut(x,32)
    
    x = Identity_shortcut(x,64,2)
    x = Identity_shortcut(x,64)
    x = Identity_shortcut(x,64)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs,outputs=outputs)
    
    model.compile(optimizer = keras.optimizers.SGD(lr =  0.1,momentum=0.9),
                  loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

model = resnet_20(shape=(32,32,3)) # or create_plain_net()
model.summary()

model.fit(
    x=x_train,
    y=y_train,
    epochs=500,
    verbose=1,
    validation_data=(x_test, y_test),
    batch_size=128
)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 32, 32, 16)   64          conv2d_19[0][0]                  
__________________________________________________________________________________________________
re_lu_19 (ReLU)                 (None, 32, 32, 16)   0           batch_normalization_19[0][0]     
____________________________________________________________________________________________

'\nmodel.fit(\n    x=x_train,\n    y=y_train,\n    epochs=500,\n    verbose=1,\n    validation_data=(x_test, y_test),\n    batch_size=128\n)'

Total params가 271,786인데 논문에는 0.27M으로 약 270000이다. 재현이 잘 되었다.